In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from random import randint
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

import re
import underthesea
from underthesea import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer 

## Crawl Comments

### Open the chrome browser

In [2]:
#Using chromedriver to open browser which install automaticly
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Open the browser with the link
browser.get("https://www.youtube.com/watch?v=6Q7mfthgMAU")



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\ADMIN\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


### Crawl comments

In [3]:
data = [] #List to store comments crawled
wait = WebDriverWait(browser, 2)

#First Scroll down and wait until everything is visible with wait.until and EC.visibility_of_element_located.
#Scroll down about 50 times to render comments
for i in range(50): 
    wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
    sleep(2) #delay randomly from 2s

#Crawl the comments by finding all the #content-text elements
comment_list = browser.find_elements(By.ID, "content-text")

#Append comments to data storage
for comment in comment_list:
    if len(data) == 200:
        break
    if comment.text not in data:
        data.append(comment.text)

In [4]:
len(data)

200

In [5]:
#Convert data list to dataframe
df = pd.DataFrame(data, columns=['comments'])
df.head()

,comments
0,ĐIỀU GÌ KHIẾN RONALDO BỚT ĐÁNG SỢ TRONG MẮT AT...
1,Đến cuối cùng thì chiếc CUP lớn nhất và quan t...
2,Không phải fan ATM nhưng thấy thắng MU là mình...
3,Ở Simone luôn có nhiệt huyết và độ quái từ khi...
4,Đúng là câu nói hằng năm với MU “đầu mùa ăn 6 ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [7]:
#Save dataframe to file .csv
df.to_csv('Crawl_comments.csv')

## Feature Extraction

In [8]:
df = pd.read_csv('./Crawl_comments.csv')

In [9]:
df.head()

,Unnamed: 0,comments
0,0,ĐIỀU GÌ KHIẾN RONALDO BỚT ĐÁNG SỢ TRONG MẮT AT...
1,1,Đến cuối cùng thì chiếc CUP lớn nhất và quan t...
2,2,Không phải fan ATM nhưng thấy thắng MU là mình...
3,3,Ở Simone luôn có nhiệt huyết và độ quái từ khi...
4,4,Đúng là câu nói hằng năm với MU “đầu mùa ăn 6 ...


**Text Preprocessing**

In [10]:
# Remove noise
def clean_noise(df, text_field, new_text_field_name):
  # convert to lower case
  df[new_text_field_name] = df[text_field].str.lower()
  # remove url
  df[new_text_field_name] = df[new_text_field_name].apply(lambda ele: re.sub(r"http\S+", "", ele))  
  # remove non-ascii and numbers
  df[new_text_field_name] = df[new_text_field_name].apply(lambda ele: re.sub("(\\W|\\d)", " ", ele))
  return df

In [11]:
clean_df = clean_noise(df,'comments','clean_comments')

In [12]:
clean_df.head(10)

,Unnamed: 0,comments,clean_comments
0,0,ĐIỀU GÌ KHIẾN RONALDO BỚT ĐÁNG SỢ TRONG MẮT AT...,điều gì khiến ronaldo bớt đáng sợ trong mắt at...
1,1,Đến cuối cùng thì chiếc CUP lớn nhất và quan t...,đến cuối cùng thì chiếc cup lớn nhất và quan t...
2,2,Không phải fan ATM nhưng thấy thắng MU là mình...,không phải fan atm nhưng thấy thắng mu là mình...
3,3,Ở Simone luôn có nhiệt huyết và độ quái từ khi...,ở simone luôn có nhiệt huyết và độ quái từ khi...
4,4,Đúng là câu nói hằng năm với MU “đầu mùa ăn 6 ...,đúng là câu nói hằng năm với mu đầu mùa ăn ...
5,5,"Chúc mừng Aletico Marid , họ là những ng xứng...",chúc mừng aletico marid họ là những ng xứng...
6,6,Chúc mừng Atletico . Chúc mừng các bạn giành v...,chúc mừng atletico chúc mừng các bạn giành v...
7,7,Hàng thủ rất ổn định trận nào cũng thủng lưới ...,hàng thủ rất ổn định trận nào cũng thủng lưới ...
8,8,"2:20 Elanga sút cận thành trúng đầu Oblak, quá...",elanga sút cận thành trúng đầu oblak quá...
9,9,Phong độ là nhất thời đẳng cấp là những gì Atl...,phong độ là nhất thời đẳng cấp là những gì atl...


**Word Segmentation with underthesea**

In [13]:
sentences=[]

all_tokens_count=0
for line in clean_df['clean_comments']:
    tokens = word_tokenize(line)
    all_tokens_count+=len(tokens)
    sentences.append(tokens)

print('all_tokens_count=',all_tokens_count)
print(len(sentences))
print(sentences)

all_tokens_count= 3991
200
[['điều', 'gì', 'khiến', 'ronaldo', 'bớt', 'đáng', 'sợ', 'trong', 'mắt', 'atletico'], ['đến', 'cuối cùng', 'thì', 'chiếc', 'cup', 'lớn', 'nhất', 'và', 'quan trọng', 'nhất', 'đối với', 'mu', 'chính', 'là', 'chiếc', 'cup', 'trong', 'lòng', 'nhm', 'đầu', 'mùa', 'nhuộm', 'đỏ', 'trời', 'âu', 'cuối', 'mùa', 'nhuộm', 'đỏ', 'hang', 'sâu', 'ngõ', 'cùng', 'thật sự', 'xúc động', 'khi', 'được', 'nhìn', 'thấy', 'các', 'cđv', 'mu', 'tri ân', 'hlv', 'diego', 'simeone', 'sau', 'trận', 'đấu'], ['không', 'phải', 'fan', 'atm', 'nhưng', 'thấy', 'thắng', 'mu', 'là', 'mình', 'rất', 'thích'], ['ở', 'simone', 'luôn', 'có', 'nhiệt huyết', 'và', 'độ', 'quái', 'từ', 'khi', 'còn', 'là', 'cầu thủ', 'có thể', 'thấy', 'simone', 'khi', 'là', 'hlv', 'thì', 'luôn', 'làm', 'cho', 'các', 'cầu thủ', 'nể', 'và', 'có', 'phần', 'sợ', 'nên', 'luôn', 'giúp', 'các', 'cầu thủ', 'thi đấu', 'đông', 'lòng', 'và', 'không', 'đòi hỏi', 'đi', 'hay', 'ở'], ['đúng', 'là', 'câu', 'nói', 'hằng', 'năm', 'với', 'mu

**Remove stopword with Vietnamese stopword dic**

In [14]:
# Read Vietnamese stopword dictionary
stop_word = pd.read_csv('./vietnamese-stopwords.txt', header=None)
stop_word.head()

,0
0,a lô
1,a ha
2,ai
3,ai ai
4,ai nấy


In [15]:
# Convert pandas to list
stop_word = stop_word[0].tolist()

In [16]:
# Remove stop words
for i in range(len(sentences)):
  sentences[i] = [word for word in sentences[i] if not word in stop_word]

In [17]:
sentences[:3]

[['ronaldo', 'bớt', 'sợ', 'mắt', 'atletico'],
 ['cup',
  'mu',
  'cup',
  'nhm',
  'đầu',
  'mùa',
  'nhuộm',
  'đỏ',
  'trời',
  'âu',
  'mùa',
  'nhuộm',
  'đỏ',
  'hang',
  'sâu',
  'ngõ',
  'xúc động',
  'cđv',
  'mu',
  'tri ân',
  'hlv',
  'diego',
  'simeone',
  'trận',
  'đấu'],
 ['fan', 'atm', 'thắng', 'mu']]

**Feature Extraction with TF-IDF**

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [19]:
# Get Feature Extraction for 200 comments with tf-idf
X = tokenizer.texts_to_matrix(sentences,mode='tfidf')
print(X[:6])
print(X.shape)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         2.10611512 0.         ... 0.         0.         0.        ]
 [0.         1.24390551 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         2.10611512 0.         ... 0.         0.         0.        ]
 [0.         1.24390551 0.         ... 0.         0.         0.        ]]
(200, 806)
